## GUI Tunning Tools for Computer Vision

In this notebook, we are going to use bqplots and ipywidget to create visual interractive tools that can be used to tune computer vision images and videos


**1. Importing packages for the GUI Tuner**

In [1]:
from ipywidgets.widgets import Label, FloatProgress, FloatSlider, Button, IntSlider
from ipywidgets.widgets import interact, interactive, fixed, interact_manual
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import cv2
import numpy as np
import bqplot as bq
import time
import threading

**2. Functions used for threshold tunning**

In [2]:
def abs_sob(img, orient,sobel_kernel=3, thresh=(0, 255)):
    low_th=thresh[0]
    high_th=thresh[1]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    sc_sob = np.uint8(255*abs_sobel/np.max(abs_sobel))
    sbinary = np.zeros_like(sc_sob)
    sbinary[(sc_sob >= low_th) & (sc_sob <= high_th)] = 1
    return sbinary
              
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    # Return the binary image
    return binary_output
              
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    # Return the binary image
    return binary_output
              
def hls_select(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

def rgb_select(img,thresh=(0,255)):
    r_channel = img[:,:,2] #noting bgr imaging
    binary_output = np.zeros_like(r_channel)
    binary_output[(r_channel > thresh[0]) & (r_channel <= thresh[1])] = 1
    return binary_output
    
              
ksize = 3 # Choose a larger odd number to smooth gradient measurements

In [52]:
# Apply each of the thresholding functions to x
def find_thresh(ksize,xlo,xhi):
    image = mpimg.imread('../test_images/test1.jpg')
    gradx = abs_sob(image, orient='x', sobel_kernel=ksize, thresh=(xlo, xhi))
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(gradx, cmap='gray')
    ax2.set_title('x Gradient', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
interact(find_thresh, ksize = IntSlider(value=11,min = 1,max = 30,step =2,description = 'kernel size',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         xlo = IntSlider(value=72,min = 0,max = 255,description = 'x low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         xhi = IntSlider(value=195,min = 0,max = 255,description = 'x high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')))
         
                                              

interactive(children=(IntSlider(value=11, description='kernel size', layout=Layout(margin='0 0 5px 0', width='…

<function __main__.find_thresh(ksize, xlo, xhi)>

In [6]:
# Apply each of the thresholding functions o x
def find_thresh(ksize,ylo,yhi):
    image = mpimg.imread('../test_images/test2.jpg')
    grady = abs_sob(image, orient='y', sobel_kernel=ksize, thresh=(ylo, yhi))
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(grady, cmap='gray')
    ax2.set_title('y Gradient', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
interact(find_thresh, ksize = IntSlider(value=1,min = 1,max = 9,step =2,description = 'kernel size',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         ylo = IntSlider(value=20,min = 0,max = 255,description = 'y low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         yhi = IntSlider(value=100,min = 0,max = 255,description = 'y high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')))
         
                                              

interactive(children=(IntSlider(value=1, description='kernel size', layout=Layout(margin='0 0 5px 0', width='4…

<function __main__.find_thresh(ksize, ylo, yhi)>

In [50]:
# Apply each of the thresholding functions o x
def find_thresh(ksize,mlo,mhi):
    image = mpimg.imread('../test_images/test1.jpg')
    gradm = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(mlo, mhi))
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(gradm, cmap='gray')
    ax2.set_title('Mag Gradient', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
interact(find_thresh, ksize = IntSlider(value=1,min = 1,max = 30,step =2,description = 'kernel size',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         mlo = IntSlider(value=103,min = 0,max = 255,description = 'm low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         mhi = IntSlider(value=224,min = 0,max = 255,description = 'm high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')))
         
                                              

interactive(children=(IntSlider(value=1, description='kernel size', layout=Layout(margin='0 0 5px 0', width='4…

<function __main__.find_thresh(ksize, mlo, mhi)>

In [16]:
# Apply each of the thresholding functions o x
def find_thresh(ksize,dlo,dhi):
    image = mpimg.imread('../test_images/test2.jpg')
    gradd = dir_threshold(image, sobel_kernel=ksize, thresh=(dlo, dhi))
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(gradd, cmap='gray')
    ax2.set_title('dir Gradient', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
interact(find_thresh, ksize = IntSlider(value=15,min = 1,max = 20,step =2,description = 'kernel size',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         dlo = FloatSlider(value=0.7,min = 0,max = 3,description = 'd low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         dhi = FloatSlider(value=1.3,min = 0,max = 3,description = 'd high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')))
         
                                              

interactive(children=(IntSlider(value=15, description='kernel size', layout=Layout(margin='0 0 5px 0', width='…

<function __main__.find_thresh(ksize, dlo, dhi)>

In [67]:
# Apply all of the thresholding functions
def find_thresh(ksize,xlo,xhi,ylo,yhi,mlo,mhi,dlo,dhi,hlo,hhi):
    image = mpimg.imread('../test_images/test5.jpg')
    gradx = abs_sob(image, orient='x', sobel_kernel=ksize, thresh=(xlo, xhi))
    grady = abs_sob(image, orient='y', sobel_kernel=ksize, thresh=(ylo, yhi))
    mag_binary = mag_thresh(image, sobel_kernel=17, mag_thresh=(mlo, mhi))
    dir_binary = dir_threshold(image, sobel_kernel=13, thresh=(dlo, dhi))
    grad_hls = hls_select(image, thresh=(hlo, hhi))
    ned = np.zeros_like(dir_binary)
    ned[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))|(grad_hls == 1)] = 1
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(ned, cmap='gray')
    ax2.set_title('Combined Gradient', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    cv2.imwrite('../output_images/test5.jpg',255*ned)
interact(find_thresh, ksize = IntSlider(value=17,min = 1,max = 30,step =2,description = 'kernel size',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         xlo = IntSlider(value=72,min = 0,max = 255,description = 'x low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         xhi = IntSlider(value=199,min = 0,max = 255,description = 'x high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         ylo = IntSlider(value=20,min = 0,max = 255,description = 'y low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         yhi = IntSlider(value=100,min = 0,max = 255,description = 'y high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         mlo = IntSlider(value=103,min = 0,max = 255,description = 'm low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         mhi = IntSlider(value=224,min = 0,max = 255,description = 'm high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         dlo = FloatSlider(value=0.7,min = 0,max = 255,description = 'd low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         dhi = FloatSlider(value=1.30,min = 0,max = 3,description = 'd high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         hlo = IntSlider(value=200,min = 0,max = 255,description = 'h low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         hhi = IntSlider(value=255,min = 0,max = 255,description = 'h high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')))
                                              

interactive(children=(IntSlider(value=17, description='kernel size', layout=Layout(margin='0 0 5px 0', width='…

<function __main__.find_thresh(ksize, xlo, xhi, ylo, yhi, mlo, mhi, dlo, dhi, hlo, hhi)>

*HLS Tresholding*

In [44]:
  # Apply each of the thresholding functions to image
def hls_thresh(ksize,hlo,hhi):
    image = mpimg.imread('../test_images/test4.jpg')
    grad_hls = hls_select(image, thresh=(hlo, hhi))
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(grad_hls, cmap='gray')
    ax2.set_title('HLS Gradient', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
interact(hls_thresh, ksize = IntSlider(value=1,min = 1,max = 9,step =2,description = 'kernel size',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         hlo = IntSlider(value=200,min = 0,max = 255,description = 'm low',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
         hhi = IntSlider(value=255,min = 0,max = 255,description = 'm high',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')))

interactive(children=(IntSlider(value=1, description='kernel size', layout=Layout(margin='0 0 5px 0', width='4…

<function __main__.hls_thresh(ksize, hlo, hhi)>